# **Slang Normalization Model**
Model ini bertugas melakukan normalisasi bahasa gaul, singkatan, typo, emoji, atau bentuk tidak baku lain menjadi bahasa Indonesia yang formal dan baku.

Contoh:

`Input: “sering sering ya min promo kaya gini”`

`Output: “sering-sering ya admin promo seperti ini”`


Tujuan Utama:
- Membersihkan teks tidak baku sebelum diproses model NLP lain
- Membantu chatbot memahami maksud pengguna dengan lebih baik
- Memperbaiki data noisy dari media sosial


# **Import Library**

In [7]:
!pip install -U bitsandbytes --quiet
!pip install accelerate transformers peft --quiet
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# **Load Model**

In [2]:
def load_model(base_model_name, lora_path):
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
    )

    mocel = PeftModel.from_pretrained(model, lora_path)
    return model, tokenizer

In [ ]:
!unzip text-norm-llama-1B-03.zip -d lora_llama

In [ ]:
model_name_decoder_only = "unsloth/Llama-3.2-1B-Instruct"
lora_path_decoder_only = "./lora_llama"

llama_model, llama_tokenizer = load_model(
    model_name_decoder_only,
    lora_path_decoder_only
)

# **Inference**

In [18]:
# --- Alpaca template ---
alpaca_prompt_norm = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Ubah teks berikut menjadi bahasa Indonesia yang baku dan sesuai dengan kata di kamus. Normalisasikan semua kata slang menjadi bentuk formal sehingga kalimat menjadi mudah dibaca.

### Input:
{}

### Response:
"""

def normalize_text(model, tokenizer, informal_text):
    device = model.device
    model.eval()
    full_prompt = alpaca_prompt_norm.format(informal_text)
    inputs = tokenizer([full_prompt], return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.3,
        top_p=0.9,
        do_sample=False,
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    return response

# **Test 1**

In [50]:
text = "gw ngantuk bgt, saatnya sleep"

In [51]:
print(normalize_text(llama_model, llama_tokenizer, text))

saya mengantuk sekali, saatnya tidur.


# **Test 2**

In [52]:
text = "gara2 pake telkomsel signal gw jelek min"

In [53]:
print(normalize_text(llama_model, llama_tokenizer, text))

gara - gara pakai telkomsel, sinyal saya jelek admin.


#**Test 3**

In [54]:
text = "monmaap ini gue juga baper kalo gitu"

In [55]:
print(normalize_text(llama_model, llama_tokenizer, text))

mohon maaf, ini saya juga bawa perasaan kalau begitu.


# **Test 4**

In [56]:
text = "lu kalo mo beli hape mending cek toko deket jalan joglo raya"

In [57]:
print(normalize_text(llama_model, llama_tokenizer, text))

kalau mau beli handphone, lebih baik periksa toko dekat jalan joglo raya.


# **Test 5**

In [58]:
text = "kesel juga gw lama2 order ojek ga ada yang nyantol"

In [59]:
print(normalize_text(llama_model, llama_tokenizer, text))

kesal juga saya lama - lama order ojek tidak ada yang menyangkut.


#**Test 6**

In [60]:
text = "jgn suka bwang2 waktu yagesya"

In [61]:
print(normalize_text(llama_model, llama_tokenizer, text))

jangan suka bwang - bwang waktu agesnya.


# **Test 7**

In [62]:
text = "goks jg ya klo tiap hari mam jengkol"

In [63]:
print(normalize_text(llama_model, llama_tokenizer, text))

goks juga ya kalau setiap hari mam jengkol.
